In [ ]:
import tensorflow as tf
import numpy as np
import os

In [ ]:
model_path = '/home/s226059/workspace/git_space/workspace/model/model_xvec_20220527-000619'
model = tf.keras.models.load_model(model_path)


In [ ]:
model.summary()

In [ ]:
dense_1 = model.get_layer('dense_1')
print(dense_1)
print(type(dense_1))

In [ ]:
import my_data_loader

spectrograms, datas = my_data_loader.set_spectrograms_and_datas()
spectrograms = spectrograms.transpose(0, 2, 1)

In [ ]:
from xvec_train import set_tf_dataset
age_th = 13
test_group = 0
classes = 59
batch_size = 1
train_specs, train_labels, valid_specs, valid_labels, test_specs, test_labels = my_data_loader.load_datas(spectrograms, datas, age_th, test_group, classes)
    
tr_ds = set_tf_dataset(train_specs, train_labels, classes=classes, batch_size=batch_size)
va_ds = set_tf_dataset(valid_specs, valid_labels, classes=classes, shuffle=False, batch_size=1)
ts_ds = set_tf_dataset(test_specs, test_labels, classes=classes, shuffle=False, batch_size=1)

In [ ]:
tr_eva = model.evaluate(tr_ds)
va_eva = model.evaluate(va_ds)
ts_eva = model.evaluate(ts_ds)

In [ ]:
print(spectrograms[0])

In [ ]:
# 中間層の出力（埋め込みベクトルの取得）
intermediate_layer_model = tf.keras.Model(inputs=model.input,
                                          outputs=model.get_layer('dense_1').output)
intermediate_output = intermediate_layer_model.predict(spectrograms)

In [ ]:
print(intermediate_output)
print(intermediate_output.shape)

In [ ]:
from xvec_train import set_tf_dataset

age_th = 13
test_group = 0
classes = 2
# log = "test/cnn " + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
epochs = 500
batch_size = 16
train_xvecs, train_labels, valid_xvecs, valid_labels, test_xvecs, test_labels = my_data_loader.load_datas(intermediate_output, datas, age_th, test_group, classes)

tr_ds = set_tf_dataset(train_xvecs, train_labels, classes=classes, batch_size=batch_size)
va_ds = set_tf_dataset(valid_xvecs, valid_labels, classes=classes, shuffle=False, batch_size=1)
ts_ds = set_tf_dataset(test_xvecs, test_labels, classes=classes, shuffle=False, batch_size=1)

In [ ]:
ds = iter(ts_ds)
d = next(ds)
print(d)

In [ ]:
from tensorflow.keras import layers, models
classes = 3

adch_model = models.Sequential()

adch_model.add(layers.Dense(512, activation='relu'))
adch_model.add(layers.BatchNormalization())
adch_model.add(layers.Dense(512, activation='relu'))
adch_model.add(layers.BatchNormalization())
if classes == 2:
    adch_model.add(layers.Dense(classes, activation="sigmoid"))
elif classes > 2:
    adch_model.add(layers.Dense(classes, activation="softmax"))

met = tf.keras.metrics.CategoricalAccuracy(False)

# adch_model.summary()
loss = tf.keras.losses.CategoricalCrossentropy()
# loss = tf.keras.losses.BinaryCrossentropy()

adch_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.000001),
            loss=loss,
            metrics=[met])

# tb = TensorBoard(log_dir=os.path.join('./logs', log), 
#                               histogram_freq=1,
#                               write_images=True,
#                               update_freq='batch',
#                               )
    
# Early Stopping
es = tf.keras.callbacks.EarlyStopping(verbose=1, patience=20)

# Model Checkpoint
# mc = tf.keras.callbacks.ModelCheckpoint(filepath = model_name,
#                                         monitor='val_loss',
#                                         verbose=1,
#                                         save_best_only=True,
#                                         save_weights_only=False,
#                                         mode='min',
#                                         period=1)

histroy = adch_model.fit(
    tr_ds,
    epochs=30,
    validation_data=va_ds,
    callbacks=[es]
)

eva = adch_model.evaluate(ts_ds)
predictions = adch_model.predict(ts_ds)

In [ ]:
from tensorflow.keras import layers, models
classes = 2

tf.keras.backend.clear_session()
adch_model = models.Sequential()

adch_model.add(layers.Dense(512, activation='relu'))
adch_model.add(layers.BatchNormalization())
adch_model.add(layers.Dense(512, activation='relu'))
adch_model.add(layers.BatchNormalization())
if classes == 2:
    adch_model.add(layers.Dense(classes, activation="sigmoid"))
elif classes > 2:
    adch_model.add(layers.Dense(classes, activation="softmax"))

met = tf.keras.metrics.CategoricalAccuracy(False)

# adch_model.summary()
# loss = tf.keras.losses.CategoricalCrossentropy()
loss = tf.keras.losses.BinaryCrossentropy()

adch_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0000001),
            loss=loss,
            metrics=[met])

# tb = TensorBoard(log_dir=os.path.join('./logs', log), 
#                               histogram_freq=1,
#                               write_images=True,
#                               update_freq='batch',
#                               )
    
# Early Stopping
es = tf.keras.callbacks.EarlyStopping(verbose=1, patience=20)

# Model Checkpoint
# mc = tf.keras.callbacks.ModelCheckpoint(filepath = model_name,
#                                         monitor='val_loss',
#                                         verbose=1,
#                                         save_best_only=True,
#                                         save_weights_only=False,
#                                         mode='min',
#                                         period=1)

histroy = adch_model.fit(
    tr_ds,
    epochs=100,
    validation_data=va_ds,
    callbacks=[es]
)

eva = adch_model.evaluate(ts_ds)
predictions = adch_model.predict(ts_ds)

In [ ]:
adch_model.summary()
tr_ds = set_tf_dataset(train_xvecs, train_labels, classes=classes, batch_size=1)
tr_eva = adch_model.evaluate(tr_ds)
va_eva = adch_model.evaluate(va_ds)
ts_eva = adch_model.evaluate(ts_ds)

In [ ]:
# モデル保存
import datetime
adch_model.save_weights('model/bi_ageth13_ts0_affinemodel'+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))